In [1]:
import time

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName('inverted-index').getOrCreate()


In [2]:
rdd = spark.sparkContext.textFile('wikipedia.dat')

In [3]:
search_words = ['JavaScript','Java','PHP','Python','C#','C++','Ruby','CSS','C','Objective-C','Shell','R','Pearl','Scala',
                'Go','Haskell','Swift','Matlab','Visual Basic','Clojure','Groovy']

search_words_lower = ['javascript','java','php','python','c#','c++','ruby','css','c','objective-c','shell','r','pearl','scala',
                'haskell','swift','matlab','visual basic','clojure','groovy']

lang_rdd = rdd.flatMap(lambda x: x.split(" ")).filter(lambda word: word in search_words).map(lambda word: (word,1))

lang_rdd_1 = rdd.flatMap(lambda x: x.split(" ")).filter(lambda word: word in search_words_lower).map(lambda word: (word,1))

In [4]:
start = time.time()

occurences_rdd = lang_rdd.groupByKey().mapValues(lambda x: sum(x))

occurences_rdd_lower = lang_rdd_1.groupByKey().mapValues(lambda x: sum(x))

print(time.time() - start)

# occurences_rdd.collect()

0.21964478492736816


In [5]:
occurrences_rdd_lower_2 = occurences_rdd.map(lambda x: (x[0].lower(),x[1]))

In [6]:
join_rdd = occurrences_rdd_lower_2.join(occurences_rdd_lower)

In [7]:
join_rdd.collect()

[('php', (806, 76)),
 ('scala', (80, 1)),
 ('pearl', (20, 3)),
 ('python', (740, 95)),
 ('javascript', (4303, 887)),
 ('ruby', (322, 7)),
 ('java', (2553, 117)),
 ('r', (453, 235)),
 ('css', (1891, 332)),
 ('matlab', (99, 25)),
 ('groovy', (32, 3)),
 ('shell', (70, 238)),
 ('c', (1754, 358)),
 ('haskell', (140, 7)),
 ('swift', (46, 30)),
 ('c++', (1250, 5)),
 ('objective-c', (344, 4)),
 ('c#', (1650, 16))]

In [8]:
result_rdd = join_rdd.mapValues(lambda x: sum([i for i in x if i is not None]))

In [9]:
sort_rdd = result_rdd.map(lambda x: (x[1],x[0])).sortByKey(ascending=False)


In [10]:
df_rdd = sort_rdd.map(lambda x: (x[1],x[0])).toDF(["Programing Language","Occurences"])

In [11]:
df_rdd.show()

+-------------------+----------+
|Programing Language|Occurences|
+-------------------+----------+
|         javascript|      5190|
|               java|      2670|
|                css|      2223|
|                  c|      2112|
|                 c#|      1666|
|                c++|      1255|
|                php|       882|
|             python|       835|
|                  r|       688|
|        objective-c|       348|
|               ruby|       329|
|              shell|       308|
|            haskell|       147|
|             matlab|       124|
|              scala|        81|
|              swift|        76|
|             groovy|        35|
|              pearl|        23|
+-------------------+----------+

